In [15]:
%load_ext autoreload
%autoreload 2
%cd ~/unilm/beit
import torch

from modeling_pretrain import beit_base_patch16_224_8k_vocab

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/austinwang/unilm/beit


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [16]:
# Input Initialization
dummy_img = torch.randn(2, 3, 224, 224)

# Torch Model

In [18]:
model = beit_base_patch16_224_8k_vocab(pretrained=False)

In [19]:
out = model(dummy_img)

TypeError: VisionTransformerForMaskedImageModeling.forward() missing 1 required positional argument: 'bool_masked_pos'